In [33]:
#таблицу базы данных и все подключения делала локально.

#подключение библиотек
import psycopg2
from psycopg2.extras import execute_values
from psycopg2 import Error
import requests
import json
from datetime import date, timedelta
import logging
import os
import csv
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email.mime.text import MIMEText
from email import encoders

In [35]:
#создаем даты вчера, сегодня, за 3 дня до последнего лога: 
yesterday_date = date.today() - timedelta(days = 1)
today_date = date.today()
three_days_early = yesterday_date - timedelta(days = 2)
#путь к логам
file_path = f"path\\to\\file\\py_log_{str(three_days_early)}.log"


In [37]:
# проверяем чтобы не было файла старше 3 дней. если есть удаляем
try:
    os.remove(file_path)
except FileNotFoundError:
    print("Файл не найден")


Файл не найден


In [39]:
# запускаем логер
logging.basicConfig(level=logging.INFO, filename=f"py_log_{str(yesterday_date)}.log",filemode="w",
                    format="%(asctime)s %(levelname)s %(message)s")

In [41]:
#класс отвечает за получение данных по API
class API_connection:
    #адрес API
    __api_url = "https://api/address"
    
    #параметры подключения
    __params = {
        'client' : 'Client_name',
        'client_key' : 'client_key',
        'start' : f'{yesterday_date}',
        'end' : f'{today_date}'
    }

    #конструктор 
    def __init__ (self):

        try:
            responce = requests.get(API_connection.__api_url, params=API_connection.__params)
            logging.info("Начато скачивание данных")
        except requests.ConnectionError as err:
            logging.info("Ошибка соединения:", err)
        except requests.exceptions.HTTPError as err:
            logging.info("Ошибка HTTP:", err)
        except requests.Timeout as err:
            logging.info("Ошибка тайм-аута:", err)
            print("Ошибка тайм-аута:", err)
        except requests.RequestException as err:
            logging.info(f"Ошибка запроса: {responce.status_code}", err)
        finally:
            logging.info(f"Скачивание данных завершено: {responce.status_code}")
            self._lst_raw_data = responce.json()
        
    @property 
    def data_getter (self):
        return self._lst_raw_data        

In [43]:
#класс отвечает за соединение с API
class API_data:
    @staticmethod
    def data_prep (lst):
        # преобразуем строку в словарь:
        for el in lst:
          for key, val in el.items():
            if key == 'passback_params':
              el[key] = json.loads(val.replace('\'','"'))
        
        # делаем словарь плоским, попутно проверяем на недостающие ключи и типы данных
        new_lst = []
        for el in lst:
          dict_temp = {}
          for key, val in el.items():
            if isinstance(val, dict):
                if 'lis_outcome_service_url' not in val.keys():
                    val.update({'lis_outcome_service_url': None})
                dict_temp.update(val)
            else:
                if isinstance(val, int): val = str(val)
                dict_temp.update({key:val})
          new_lst.append(dict_temp)
        return new_lst


In [45]:
# подключение к БД по паттерну Синглтон
class DatabaseConnection:
  __instance = None
  __HOST = 'host'
  __PORT = 'port_number'
  __DATABASE = 'database_name'
  __USER = 'user'
  __PASSWORD = 'your_password'

  @staticmethod
  def get_instance():
    if not DatabaseConnection.__instance:
      DatabaseConnection()
    return DatabaseConnection.__instance

  def __init__(self):
    if DatabaseConnection.__instance:
      raise Exception("Этот класс является Singleton")
      logging.info('Ошибка повторное создание экземпляра Singleton')
    else:
      self.connection = psycopg2.connect(host = DatabaseConnection.__HOST, 
                                         port = DatabaseConnection.__PORT, 
                                         database = DatabaseConnection.__DATABASE, 
                                         user = DatabaseConnection.__USER, 
                                         password = DatabaseConnection.__PASSWORD)
      DatabaseConnection.__instance = self
      logging.info('Соединение с базой данных LMS_SF установлено')
        
  def get_connection(self):
    return self.connection 
    

In [47]:
#здесь работаем с базой данных при необходимости можно дополнить другими методами для работы с БД
class DB_queries:

    def __init__ (self, connection):
        self._connection = connection
        self._cursor = connection.cursor()

    
    def post_values(self, lst):
        #преобразуем в кортежи те словари, которые получены в классе API_data
        val_to_insert = [tuple(el.values()) for el in lst]
        try:
            execute_values(self._cursor,'INSERT INTO users_activity (user_id,  oauth_consumer_key, lis_result_sourcedid, lis_outcome_service_url, is_correct, attempt_type, created_at) VALUES %s', val_to_insert)
            self._connection.commit()
            logging.info("Данные записаны в базу LMS_SF")
        except (Exception, Error) as err:
            logging.info("Ошибка при работе с базой LMS_SF", err)
        finally:
            if self._connection:
                self._cursor.close()
                self._connection.close()
                logging.info("Соединение с базой LMS_SF закрыто")
        

In [49]:
# у меня не получилось поработать с API гуглтаблиц из-за проблем с аккаунтом GoogleCloud, 
# поэтому я приняла решение сохранять результаты аналитики по пользователям и отправлять его на почту  

In [51]:
#класс для работы с формированием и тправкой сообщения
class Analys_and_send:
    #атрибуты сервера почты
    #NB! убрать персоналку для открытого доступа
    __address_from = 'your_email@mail.ru'
    __server_pass = 'server_pass'
    
    def __init__ (self, filepath):
        #путь к файлу с аналитикой, который будем записывать и пересылать
        self.__file_path = filepath        

    # метод собирает аналитику по пользователям и записывает в файл
    def create_file(self, lst_base):
        # создаем словарь, который будем записывать в файл
        dct_analys = {'Всего попыток(submit)': len(list(filter(lambda x: x.get('attempt_type') == 'submit', lst_base))),
                      'Из них удачных': len(list(filter(lambda x: x.get('is_correct') == '1', lst_base))), 
                      'Запусков кода(run)': len(list(filter(lambda x: x.get('attempt_type') == 'run', lst_base))),
                      'Уникальных пользователей': len(set(map(lambda x: x.get('lti_user_id'), lst_base))),
                      'Дата': yesterday_date
                     }
        #шапка таблицы
        columns = list(dct_analys.keys())
        # запись в файл csv если файл существует или нет
        if os.path.exists(self.__file_path):
            with open(self.__file_path, 'a') as f:
                writer = csv.DictWriter(f,  delimiter=',',fieldnames=columns)
                try:
                    writer.writerow(dct_analys)
                    logging.info(f"Данные записаны в файл {self.__file_path}")
                except Exception as err:
                    logging.info(f"Ошибка записи в файл {self.__file_path}", err)
        
        else:
            with open(self.__file_path, 'w') as f:        
                writer = csv.DictWriter(f,  delimiter=',', fieldnames=columns)
                try:
                    writer.writeheader()
                    writer.writerow(dct_analys)
                    logging.info(f"Данные записаны в файл {self.__file_path}")
                except Exception as err:
                    logging.info(f"Ошибка записи в файл {self.__file_path}", err)

    #допустим список рассылки хранится в файле csv 
    @staticmethod
    def get_mailing_list(filepath_mailing_list):
        emails = []
        with open(filepath_mailing_list, 'r') as f:
          reader = csv.DictReader(f, delimiter=',')
          for row in reader:
            emails.append(row['email'])
        return emails
        
    # метод для отправки письма со вложением    
    def mail_sender (self, filepath_mailing_list):
        #чтобы не городить огород в отправке сохраним значения атрибутов 
        address_from = Analys_and_send.__address_from
        server_pass = Analys_and_send.__server_pass        
        address_to = Analys_and_send.get_mailing_list(filepath_mailing_list)
          
        # откидываем все директории до имени файла 
        basename = os.path.basename(self.__file_path)
        
        #создаем объект многочастное сообщение
        msg = MIMEMultipart()
        
        #реквизиты сообщения
        msg['From'] = address_from
        msg['To'] = ', '. join(address_to)
        msg['Subject'] = f'Аналитика по платформе за {yesterday_date}'
        
        #собственно тело письма
        message = f"Добрый день! Обновленный файл с аналитикой по результатам пользователей во вложеннии. Добавлены результаты за {yesterday_date}"
        
        #добавляем тело письма в сообщение
        msg.attach(MIMEText(message))
        
        #Формируем вложение к письму
        part = MIMEBase('application', "octet-stream")
        part.set_payload(open(self.__file_path,"rb").read() )
        encoders.encode_base64(part)
        part.add_header('Content-Disposition', 'attachment; filename="%s"' % basename)
        
        #добавляем вложение
        msg.attach(part)
        
        #отправляем сообщение
        try:
            server = smtplib.SMTP_SSL('smtp.mail.ru',465)
            server.set_debuglevel(True)
            server.login(address_from, server_pass)
            server.sendmail(address_from, address_to, msg.as_string())        
            logging.info("Письмо успешно отправлено")
        except smtplib.SMTPException as err:
            logging.info("Ошибка: Невозможно отправить сообщение", err)
        finally:
            server.quit()
            logging.info("Соединение с почтовым сервером закрыто")
        


In [53]:
#получаем данные по API
data_from_API = API_connection()

#проводим валидацию данных
lst_base = API_data().data_prep(data_from_API.data_getter)

# создаем соединение с базой данных
db = DatabaseConnection()

# создаем объект для работы с БД
db_insert = DB_queries(db.get_connection())

#переносим данные в БД
db_insert.post_values(lst_base)

#создаем объект для работы с созданием файлами
sender = Analys_and_send('analit.csv')

#создаем или дозаписываем в файл аналитику по пользователям
sender.create_file(lst_base)

# отправляем сообщение
sender.mail_sender('mail_list.csv')

send: 'ehlo [192.168.56.1]\r\n'
reply: b'250-smtp.mail.ru\r\n'
reply: b'250-SIZE 73400320\r\n'
reply: b'250-8BITMIME\r\n'
reply: b'250-DSN\r\n'
reply: b'250-SMTPUTF8\r\n'
reply: b'250 AUTH PLAIN LOGIN XOAUTH2\r\n'
reply: retcode (250); Msg: b'smtp.mail.ru\nSIZE 73400320\n8BITMIME\nDSN\nSMTPUTF8\nAUTH PLAIN LOGIN XOAUTH2'
send: 'AUTH PLAIN AGFsZXhhbmRyYV9tYWx0c2V2YUBiay5ydQBISEtieHVkTEhBMFdaQ3B2RUUwUw==\r\n'
reply: b'235 Authentication succeeded\r\n'
reply: retcode (235); Msg: b'Authentication succeeded'
send: 'mail FROM:<alexandra_maltseva@bk.ru> size=1118\r\n'
reply: b'250 OK\r\n'
reply: retcode (250); Msg: b'OK'
send: 'rcpt TO:<lexie70rus@gmail.com>\r\n'
reply: b'250 Accepted\r\n'
reply: retcode (250); Msg: b'Accepted'
send: 'rcpt TO:<rin_14@inbox.ru>\r\n'
reply: b'250 Accepted\r\n'
reply: retcode (250); Msg: b'Accepted'
send: 'data\r\n'
reply: b'354 Enter message, ending with "." on a line by itself\r\n'
reply: retcode (354); Msg: b'Enter message, ending with "." on a line by itself